In [2]:
import os
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
import numpy as np
import joblib
import glob

Using TensorFlow backend.


In [3]:
base_model = VGG19(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('flatten').output)

In [4]:
def get_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    flatten = model.predict(x)
    return list(flatten[0])

In [5]:
chunks = joblib.load("chunks.joblib")

In [6]:
def extract(audio_file):
    name=os.path.basename(audio_file)
    name=os.path.splitext(name)[0]
    label = int(name[7:8]) - 1
    
    return label

In [7]:
dir_name = glob.glob("../feats/vgg_amp_plts/*.png")

In [8]:
len(dir_name)

5252

In [9]:
X = []
y = []

In [10]:
X_ = joblib.load("X_plots_feats.joblib")

EOFError: 

In [9]:
for item in dir_name:
    X.append(get_features(item))
    y.append(extract(item))

In [ ]:
joblib.dump(X, "X_plots_feats.joblib")
joblib.dump(y, "y_plots_feats.joblib")

X_ = joblib.load("X_plots_feats.joblib")
y_ = joblib.load("y_plots_feats.joblib")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time
from xgboost import XGBClassifier

X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.33, random_state=42)

start_time = time.time()

model_XGB = XGBClassifier(learning_rate = 0.1,
                      n_estimators = 1000,
                      max_depth = 8,
                      min_child_weight=3,
                      subsample = 0.8,
                      colsample_bytree = 0.8,
                      gamma = 0
                     )
model_XGB.fit(X_train, y_train)

y_pred = model_XGB.predict(X_test)
predictions = [round(value) for value in y_pred]

print("--- Model trained. Training time: %s seconds ---" % (time.time() - start_time))

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

joblib.dump(model_XGB, "../Models/VGG.joblib.dat")